<h1><center>🌏Basic and Useful 📊EDA For Beginner🛰</center></h1>

<center><img src="https://www.nasa.gov/sites/default/files/styles/full_width_feature/public/images/297755main_GPN-2001-000009_full.jpg"></center>

<h3><I>“Look again at that dot. That's here. That's home. That's us. On it everyone you love, everyone you know, everyone you ever heard of, every human being who ever was, lived out their lives. The aggregate of our joy and suffering, thousands of confident religions, ideologies, and economic doctrines, every hunter and forager, every hero and coward, every creator and destroyer of civilization, every king and peasant, every young couple in love, every mother and father, hopeful child, inventor and explorer, every teacher of morals, every corrupt politician, every "superstar," every "supreme leader," every saint and sinner in the history of our species lived there-on a mote of dust suspended in a sunbeam."</I></h3>
<h3><div align="right">- Carl Sagan -</div></h3>

<h2><center>If you think this notebook useful, support with an upvote👍</center></h2>
<h2><center>Upvote is Free 😊</center></h2>

# 1. Data Load

In [ ]:
import numpy as np 
import pandas as pd 

DATA_PATH = '/kaggle/input/spaceship-titanic/'
train = pd.read_csv(DATA_PATH + 'train.csv')
test = pd.read_csv(DATA_PATH + 'test.csv')
submission = pd.read_csv(DATA_PATH + 'sample_submission.csv')

# 2. Take a Look at Data

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
submission.head()

## Visualize Null value

In [ ]:
import missingno as msno

# Convert -1 to np.nan in the train data copy
train_copy = train.copy()

# visualize null value
msno.matrix(df=train_copy, figsize=(15, 4));

In [ ]:
# visualize null value with bar
msno.bar(df=train_copy, figsize=(15, 4));

## Function to Downcast
Downcast is a great skill to compress data size which helps to save memory, speed up model training.



In [ ]:
def downcast(df, verbose=True):
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        dtype_name = df[col].dtype.name
        if dtype_name == 'object':
            pass
        elif dtype_name == 'bool':
            df[col] = df[col].astype('int8')
        elif dtype_name.startswith('int') or (df[col].round() == df[col]).all():
            df[col] = pd.to_numeric(df[col], downcast='integer')
        else:
            df[col] = pd.to_numeric(df[col], downcast='float')
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose:
        print('{:.1f}% Compressed'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [ ]:
train = downcast(train)

## Function to create feature summary

In [ ]:
def resumetable(df):
    print(f'Shape : {df.shape}')
    summary = pd.DataFrame(df.dtypes, columns=['Data Type'])
    summary = summary.reset_index()
    summary = summary.rename(columns={'index': 'Feature'})
    summary['Num of null'] = df.isnull().sum().values
    summary['Num of unique'] = df.nunique().values
    summary['First value'] = df.loc[0].values
    summary['Second value'] = df.loc[1].values
    summary['Third value'] = df.loc[2].values
    return summary

In [ ]:
resumetable(train)

- `PassengerId` is ID
- `Transported` is Target Value
- `HomePlanet`, `CryoSleep`, `Cabin`, `Destination`, `VIP`, `Name`, are categorical data.
- `Age`, `RoomService`, `FoodCourt`, `ShoppingMall`, `Spa`, `VRDeck` are numerical data.

In [ ]:
train.describe()

# 3-1. Data Visualization: Simple Distribution

In [ ]:
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

plt.style.use('fivethirtyeight')

### Function to write percent at the top of the bar graph

In [ ]:
def write_percent(ax, total_size):
    '''Traverse the figure object and display the ratio at the top of the bar graph.'''
    for patch in ax.patches:
        height = patch.get_height() # Figure height (number of data)
        width = patch.get_width() # Figure width
        left_coord = patch.get_x() # The x-axis position on the left edge of the figure
        percent = height/total_size*100 # percent
        
        # Type text in the (x, y) coordinates
        ax.text(x=left_coord + width/2.0, # x-axis position
                y=height + total_size*0.001, # y-axis position
                s=f'{percent:1.1f}%', # Text
                ha='center') # in the middle

## Distribution of Target Value

In [ ]:
ax = sns.countplot(x='Transported', data=train)
write_percent(ax, len(train)) 
ax.set_title('Transported Counts');

## Distribution of Categorical Data
- HomePlanet, CryoSleep, Cabin, Destination, VIP, Name are categorical data.
- Cabin and Name features have lots of unique values so that I only plot HomePlanet, CryoSleep, Destination, VIP features

In [ ]:
mpl.rc('font', size=13) # Set font size

ax = sns.countplot(x='HomePlanet', data=train)
write_percent(ax, len(train)) 
ax.set_title('HomePlanet Counts');

In [ ]:
ax = sns.countplot(x='CryoSleep', data=train)
write_percent(ax, len(train)) 
ax.set_title('CryoSleep Counts');

In [ ]:
ax = sns.countplot(x='Destination', data=train)
write_percent(ax, len(train)) 
ax.set_title('Destination Counts');

In [ ]:
ax = sns.countplot(x='VIP', data=train)
write_percent(ax, len(train)) 
ax.set_title('VIP Counts');

## Distribution of Numerical Features
- Age, RoomService, FoodCourt, ShoppingMall, Spa, VRDeck are numerical data


### Age Feature

In [ ]:
plt.figure(figsize=(8, 4)) # Set figure size

ax = sns.histplot(x='Age', data=train)
ax.set_title('Age Counts');

### Last numerical Features
- The five numerical features('RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck') have too many unqiue values, so I divided the bins and plot the bar graph.

In [ ]:
import matplotlib.gridspec as gridspec

numerical_features = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck'] # Numerical Features except Age

train_copy = train.copy()

plt.figure(figsize=(20, 16))                # Set Figure Size
grid = gridspec.GridSpec(3, 2)              # Create GridSpec Object
plt.subplots_adjust(wspace=0.2, hspace=0.4) # Set the space between subplots.

for idx, numerical_feature in enumerate(numerical_features):
    # Divide the value into 5 bins
    train_copy[numerical_feature] = pd.cut(train_copy[numerical_feature], 5)

    ax = plt.subplot(grid[idx])                # Set the subplot to draw the distribution
    sns.barplot(x=numerical_feature, y='Transported', data=train_copy, palette='Set2', ax=ax)
    ax.tick_params(axis='x', labelrotation=10) # Rotate x-axis labels

# 3-2. Other Data Visualizations

## Distribution With True Ratio

In [ ]:
def get_crosstab(df, feature):
    crosstab = pd.crosstab(df[feature], df['Transported'], normalize='index')*100
    crosstab = crosstab.reset_index()
    return crosstab

In [ ]:
def plot_pointplot(ax, feature, crosstab):
    ax2 = ax.twinx() # Create a new axis that shares the x-axis and not the y-axis
    # Plot a point plot on a new axis.
    ax2 = sns.pointplot(x=feature, y=1, data=crosstab,
                        order=crosstab[feature].values, # Order of point-plot
                        color='black',                  # 
                        legend=False)                   # No legend
    ax2.set_ylim(crosstab[1].min()-5, crosstab[1].max()*1.1) # Set y-axis range
    ax2.set_ylabel('Target 1 Ratio(%)')

In [ ]:
def plot_cat_dist_with_true_ratio(df, features, num_rows, num_cols, 
                                  size=(17, 10)):
    plt.figure(figsize=size)  
    grid = gridspec.GridSpec(num_rows, num_cols)
    plt.subplots_adjust(wspace=0.45, hspace=0.3) 
    
    for idx, feature in enumerate(features): 
        ax = plt.subplot(grid[idx])
        crosstab = get_crosstab(df, feature) # Create CrossTab

        # Plot the target value distribution count plot on the ax axis
        sns.countplot(x=feature, data=df,
                      order=crosstab[feature].values,
                      color='skyblue',
                      ax=ax)

        write_percent(ax, len(df)) 
       
        plot_pointplot(ax, feature, crosstab) 
        
        ax.set_title(f'{feature} Distribution') # Set plot title

In [ ]:
mpl.rc('font', size=13) # Set font size

nom_features = ['HomePlanet', 'CryoSleep', 'Destination', 'VIP']
plot_cat_dist_with_true_ratio(train, nom_features, num_rows=2, num_cols=2)

## Target Ration by Features

In [ ]:
def plot_target_ratio_by_features(df, features, num_rows, num_cols, 
                                  size=(18, 8)):
    mpl.rc('font', size=13) 
    plt.figure(figsize=size)                     # Set Figure Size
    grid = gridspec.GridSpec(num_rows, num_cols) # Set subplots
    plt.subplots_adjust(wspace=0.3, hspace=0.3)  # Set subplots space

    for idx, feature in enumerate(features):
        ax = plt.subplot(grid[idx])
        # Plot a bar-graph of the target value ratio by unique value on the ax-axis.
        sns.barplot(x=feature, y='Transported', data=df, palette='Set2', ax=ax)

In [ ]:
cat_features = ['HomePlanet', 'CryoSleep', 'Destination', 'VIP']

plot_target_ratio_by_features(train, cat_features, 2, 2) # 2 by 2